In [15]:
df = pd.read_csv("./bulkdata/final_descriptions_with_scores.csv")

# 최종 DataFrame에서 description 컬럼만 선택
final_description_df = df[['description']]

# CSV 파일로 저장 (파일 경로는 필요에 따라 수정)
final_description_df.to_csv("./bulkdata/descriptions.csv", index=False)

print("CSV 파일 저장 완료")


CSV 파일 저장 완료


In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
import io

# 1. 학습 데이터 재정의 (사용자 레이블 및 피드백)
user_labels_data = """
❄ 에어컨을 26도로 설정하고 필요한 시간에만 켠다.,1
💡 집 안의 모든 조명을 형광등으로 유지한다.,-1
🧺 세탁물을 모아서 세탁기를 주 2~3회만 돌린다.,1
🔌 전기밥솥 보온 기능을 하루 종일 켜둔다.,-1
🔌 사용하지 않는 전자제품의 플러그를 뽑아둔다.,1
🥘 음식물 찌꺼기를 물로 씻어 하수구에 흘려보낸다.,-1
💧 세제를 사용한 걸레를 흐르는 물로 오래 헹군다.,-1
💧 기름 묻은 후라이팬을 키친타월로 닦은 후 설거지한다.,1
🗑 화장실에 물티슈나 기저귀를 버린다.,-1
💧 남은 약이나 폐기물을 약국 또는 지정 수거함에 버린다.,1
🛢 요리 중 냄비 뚜껑을 덮고 조리한다.,1
🔥 겨울철 보일러를 하루 종일 틀어놓고 외출한다.,-1
"""

# 데이터프레임 생성
user_df = pd.read_csv(io.StringIO(user_labels_data), header=None, names=["description", "score"])

# 학습 데이터 결합 후, NaN 값 제거
updated_train_data = updated_train_data.dropna(subset=["user_score"])

# 2. TF-IDF 벡터화 및 Ridge 회귀 모델 재학습
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(updated_train_data["description"])
y_train = updated_train_data["user_score"].astype(float)

reg_model = Ridge()
reg_model.fit(X_train, y_train)

# 3. CSV 파일 불러오기 및 모델 예측
df_reloaded = pd.read_csv("./bulkdata/descriptions.csv")

# description 칼럼 기반 예측 수행
X_all = vectorizer.transform(df_reloaded["description"])
reg_predictions = reg_model.predict(X_all)

# 이진 긍/부정 판단: 0보다 크면 1(긍정), 그렇지 않으면 -1(부정)
df_reloaded["PN"] = [1 if score > 0 else -1 for score in reg_predictions]

# 결과 미리 보기
print(df_reloaded.head(100))


                            description  PN
0         ❄ 에어컨을 26도로 설정하고 필요한 시간에만 켠다.   1
1             💡 집 안의 모든 조명을 형광등으로 유지한다.  -1
2          🧺 세탁물을 모아서 세탁기를 주 2~3회만 돌린다.   1
3              🔌 전기밥솥 보온 기능을 하루 종일 켜둔다.  -1
4            🔌 사용하지 않는 전자제품의 플러그를 뽑아둔다.   1
5          🥘 음식물 찌꺼기를 물로 씻어 하수구에 흘려보낸다.  -1
6          💧 세제를 사용한 걸레를 흐르는 물로 오래 헹군다.  -1
7       💧 기름 묻은 후라이팬을 키친타월로 닦은 후 설거지한다.   1
8                 🗑 화장실에 물티슈나 기저귀를 버린다.  -1
9      💧 남은 약이나 폐기물을 약국 또는 지정 수거함에 버린다.   1
10               🛢 요리 중 냄비 뚜껑을 덮고 조리한다.   1
11          🔥 겨울철 보일러를 하루 종일 틀어놓고 외출한다.  -1
12     🔥 보일러 온도는 낮추고, 온수는 필요한 만큼만 사용한다.   1
13         🛢 가스레인지에 불꽃이 냄비 바깥으로 퍼지게 켠다.   1
14  🛢 가스 누출 점검을 정기적으로 하고, 오래된 호스를 교체한다.   1
15         🧺 세탁물을 모아서 세탁기를 주 2~3회만 돌린다.   1
16            💡 집 안의 모든 조명을 형광등으로 유지한다.  -1
17                        🧺 매일 세탁기를 돌린다   1
18             🔌 전기밥솥 보온 기능을 하루 종일 켜둔다.  -1
19                      💡 불필요한 조명을 꺼둔다.  -1
20                      ❄ 냉장고 문을 자주 연다.   1
21                ❄ 에어컨 온도를 항상 낮

In [19]:
import pandas as pd

# 긍정 단어 100개 (각 단어는 최대 3글자)
positive_words_100 = [
    "절약", "효율", "친환", "재활", "보존", "절감", "최소", "지속", "절전", "저탄",
    "개선", "정화", "순환", "재생", "녹색", "청정", "산림", "해양", "자연", "중립",
    "다양", "에코", "혁신", "신재", "녹성", "회복", "도시", "경영", "투자", "정책",
    "자원", "소비", "의식", "스마트", "녹기", "물절", "친농", "자립", "실천", "행동",
    "지구", "환기", "환원", "감소", "조절", "에너", "산업", "녹산", "바람", "태양",
    "수력", "풍력", "전환", "저배", "녹전", "친건", "친소", "친화", "녹업", "신기",
    "산보", "해보", "친업", "전력", "녹율", "녹원", "친도", "녹지", "청원", "재순",
    "친생", "환산", "탄감", "녹실", "녹환", "에녹", "신환", "자환", "청환", "녹창",
    "에친", "효보", "에보", "산청", "녹청", "에신", "자경", "친자", "산지", "해지",
    "자지", "에경", "에정", "재개", "녹개", "순보", "에순", "친합", "자합", "녹합"
]

# 부정 단어 100개 (각 단어는 최대 3글자)
negative_words_100 = [
    "낭비", "과다", "무분", "상시", "불필", "과도", "과소", "오염", "폐기", "파괴",
    "배출", "독성", "부적", "비효", "무절", "자낭", "남용", "무용", "소음", "미세",
    "화석", "비재", "독물", "온실", "과열", "불균", "불안", "폐물", "에낭", "무계",
    "파적", "해로", "위협", "불안", "부실", "부정", "자형", "부실운", "오남", "무효",
    "독배", "폐증", "소과", "불친", "비도", "위험", "낭성", "비합", "대오", "수오",
    "토오", "소공", "오물", "해오", "기악", "불투", "산폐", "산파", "자파", "대질",
    "자남", "화출", "유해", "폐수", "악취", "기유", "생파", "피해", "쓰범", "미폭",
    "독폐", "불법", "불광", "비위", "무책", "폐부", "온과", "온난", "공해", "규미",
    "피증", "재불", "무추", "쓰문", "도열", "온변", "위기", "생행", "불합", "비불",
    "부퇴", "폐악", "불폐", "비낭", "비오", "과배", "부오", "비폐", "지오", "산오"
]

def lexicon_score(text, pos_list, neg_list):
    """텍스트 내에 포함된 긍정 단어와 부정 단어의 개수를 세어 점수를 산출합니다."""
    pos_count = sum(1 for word in pos_list if word in text)
    neg_count = sum(1 for word in neg_list if word in text)
    return pos_count - neg_count

# CSV 파일 불러오기 (파일 경로를 실제 경로에 맞게 수정하세요)
df = pd.read_csv("./bulkdata/descriptions.csv")

# 각 문장에 대해 사전 기반 점수 계산 (lex_score)
df["lex_score"] = df["description"].apply(lambda x: lexicon_score(x, positive_words_100, negative_words_100))

# 결과 CSV 파일로 저장 (예: descriptions_with_lexicon.csv)
df.to_csv("./bulkdata/descriptions_with_lexicon.csv", index=False)

print("적용 완료: descriptions_with_lexicon.csv")


적용 완료: descriptions_with_lexicon.csv


In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 긍정 단어 100개 (각 단어는 최대 3글자)
positive_words_100 = [
    "절약", "효율", "친환", "재활", "보존", "절감", "최소", "지속", "절전", "저탄",
    "개선", "정화", "순환", "재생", "녹색", "청정", "산림", "해양", "자연", "중립",
    "다양", "에코", "혁신", "신재", "녹성", "회복", "도시", "경영", "투자", "정책",
    "자원", "소비", "의식", "스마트", "녹기", "물절", "친농", "자립", "실천", "행동",
    "지구", "환기", "환원", "감소", "조절", "에너", "산업", "녹산", "바람", "태양",
    "수력", "풍력", "전환", "저배", "녹전", "친건", "친소", "친화", "녹업", "신기",
    "산보", "해보", "친업", "전력", "녹율", "녹원", "친도", "녹지", "청원", "재순",
    "친생", "환산", "탄감", "녹실", "녹환", "에녹", "신환", "자환", "청환", "녹창",
    "에친", "효보", "에보", "산청", "녹청", "에신", "자경", "친자", "산지", "해지",
    "자지", "에경", "에정", "재개", "녹개", "순보", "에순", "친합", "자합", "녹합"
]

# 부정 단어 100개 (각 단어는 최대 3글자)
negative_words_100 = [
    "낭비", "과다", "무분", "상시", "불필", "과도", "과소", "오염", "폐기", "파괴",
    "배출", "독성", "부적", "비효", "무절", "자낭", "남용", "무용", "소음", "미세",
    "화석", "비재", "독물", "온실", "과열", "불균", "불안", "폐물", "에낭", "무계",
    "파적", "해로", "위협", "불안", "부실", "부정", "자형", "부실운", "오남", "무효",
    "독배", "폐증", "소과", "불친", "비도", "위험", "낭성", "비합", "대오", "수오",
    "토오", "소공", "오물", "해오", "기악", "불투", "산폐", "산파", "자파", "대질",
    "자남", "화출", "유해", "폐수", "악취", "기유", "생파", "피해", "쓰범", "미폭",
    "독폐", "불법", "불광", "비위", "무책", "폐부", "온과", "온난", "공해", "규미",
    "피증", "재불", "무추", "쓰문", "도열", "온변", "위기", "생행", "불합", "비불",
    "부퇴", "폐악", "불폐", "비낭", "비오", "과배", "부오", "비폐", "지오", "산오"
]

def lexicon_score(text, pos_list, neg_list):
    """텍스트 내 긍정/부정 단어 개수를 세어 점수를 반환합니다."""
    pos_count = sum(1 for word in pos_list if word in text)
    neg_count = sum(1 for word in neg_list if word in text)
    return pos_count - neg_count

# CSV 파일 불러오기 (파일 경로를 실제 경로에 맞게 수정)
df = pd.read_csv("./bulkdata/descriptions.csv")

# 각 문장에 대해 사전 기반 점수 계산 (lex_score)
df["lex_score"] = df["description"].apply(lambda x: lexicon_score(x, positive_words_100, negative_words_100))

# sentence-transformers 모델 로드 (다국어 지원 모델 사용)
# 한국어의 경우, 'distiluse-base-multilingual-cased' 또는 'paraphrase-multilingual-MiniLM-L12-v2'와 같이 다국어 모델을 권장합니다.
model = SentenceTransformer('distiluse-base-multilingual-cased')

# 긍정 단어 및 부정 단어의 임베딩 계산 (모델에 존재하는 단어 모두 사용)
pos_embs = [model.encode(word) for word in positive_words_100]
avg_pos_emb = np.mean(pos_embs, axis=0) if pos_embs else None

neg_embs = [model.encode(word) for word in negative_words_100]
avg_neg_emb = np.mean(neg_embs, axis=0) if neg_embs else None

def assign_label_sentence_transformers(text):
    """문장의 임베딩을 구한 후, 긍정/부정 평균 임베딩과의 코사인 유사도를 비교하여 라벨을 반환합니다."""
    text_emb = model.encode(text)
    pos_sim = cosine_similarity(text_emb.reshape(1, -1), avg_pos_emb.reshape(1, -1))[0][0] if avg_pos_emb is not None else 0
    neg_sim = cosine_similarity(text_emb.reshape(1, -1), avg_neg_emb.reshape(1, -1))[0][0] if avg_neg_emb is not None else 0
    return 1 if pos_sim > neg_sim else -1

def compute_final_label(row):
    # 사전 기반 점수가 0이 아니면 그 결과 사용, 0이면 sentence-transformers 기반 유사도 판단
    if row["lex_score"] != 0:
        return 1 if row["lex_score"] > 0 else -1
    else:
        return assign_label_sentence_transformers(row["description"])

# 최종 라벨(PN_final) 부여
df["PN_final"] = df.apply(compute_final_label, axis=1)

# 결과 확인 (상위 20행 출력)
print(df.head(20))

# 결과를 CSV 파일로 저장
df.to_csv("./bulkdata/descriptions_with_PN_final.csv", index=False)
print("최종 결과 저장 완료: descriptions_with_PN_final.csv")



C:\Users\SSAFY\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\SSAFY\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see http

                            description  lex_score  PN_final
0         ❄ 에어컨을 26도로 설정하고 필요한 시간에만 켠다.          0         1
1             💡 집 안의 모든 조명을 형광등으로 유지한다.          0         1
2          🧺 세탁물을 모아서 세탁기를 주 2~3회만 돌린다.          0         1
3              🔌 전기밥솥 보온 기능을 하루 종일 켜둔다.          0         1
4            🔌 사용하지 않는 전자제품의 플러그를 뽑아둔다.          0        -1
5          🥘 음식물 찌꺼기를 물로 씻어 하수구에 흘려보낸다.          0        -1
6          💧 세제를 사용한 걸레를 흐르는 물로 오래 헹군다.          0         1
7       💧 기름 묻은 후라이팬을 키친타월로 닦은 후 설거지한다.          0         1
8                 🗑 화장실에 물티슈나 기저귀를 버린다.          0        -1
9      💧 남은 약이나 폐기물을 약국 또는 지정 수거함에 버린다.         -1        -1
10               🛢 요리 중 냄비 뚜껑을 덮고 조리한다.          0        -1
11          🔥 겨울철 보일러를 하루 종일 틀어놓고 외출한다.          0        -1
12     🔥 보일러 온도는 낮추고, 온수는 필요한 만큼만 사용한다.          0         1
13         🛢 가스레인지에 불꽃이 냄비 바깥으로 퍼지게 켠다.          0        -1
14  🛢 가스 누출 점검을 정기적으로 하고, 오래된 호스를 교체한다.          0         1
15         🧺 세탁물을 모아서 세탁

In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

# 1. CSV 파일에서 트레이닝 데이터 로드
#    파일에는 'description'과 'PN' 컬럼이 있다고 가정합니다.
df_train = pd.read_csv("./bulkdata/descriptions_with_PN.csv")
print("데이터셋 컬럼:", df_train.columns)

# 2. 사전학습된 SentenceTransformer 모델 로드 (다국어 지원)
model = SentenceTransformer('distiluse-base-multilingual-cased')

# 3. 학습 데이터에 대해 임베딩 생성
#    각 문장을 모델을 이용해 벡터화합니다.
descriptions = df_train['description'].tolist()
X_train = model.encode(descriptions)
y_train = df_train['PN'].values  # 라벨: 1 (긍정), -1 (부정)

# 4. 간단한 분류기(로지스틱 회귀) 학습
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# 5. 새로운 문장을 입력받아 예측 결과를 반환하는 함수 정의
def predict_sentiment(sentence):
    """
    입력된 문장에 대해 긍정(1) 또는 부정(-1) 라벨을 반환합니다.
    
    Parameters:
        sentence (str): 예측할 문장
        
    Returns:
        int: 예측된 라벨 (1 또는 -1)
    """
    embedding = model.encode([sentence])
    pred = clf.predict(embedding)
    return int(pred[0])

# 예시: 테스트 문장 예측
test_sentence = "이 제품은 정말 효율적이고 환경에도 좋아요."
result = predict_sentiment(test_sentence)
print("예측 결과:", result)

데이터셋 컬럼: Index(['description', 'score', 'PN'], dtype='object')
예측 결과: -1


In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sentence_transformers import SentenceTransformer

# ── 1. 기본 단어 리스트 ──
positive_words_100 = [
    "절약", "효율", "친환", "재활", "보존", "절감", "최소", "지속", "절전", "저탄",
    "개선", "정화", "순환", "재생", "녹색", "청정", "산림", "해양", "자연", "중립",
    "다양", "에코", "혁신", "신재", "녹성", "회복", "도시", "경영", "투자", "정책",
    "자원", "소비", "의식", "스마트", "녹기", "물절", "친농", "자립", "실천", "행동",
    "지구", "환기", "환원", "감소", "조절", "에너", "산업", "녹산", "바람", "태양",
    "수력", "풍력", "전환", "저배", "녹전", "친건", "친소", "친화", "녹업", "신기",
    "산보", "해보", "친업", "전력", "녹율", "녹원", "친도", "녹지", "청원", "재순",
    "친생", "환산", "탄감", "녹실", "녹환", "에녹", "신환", "자환", "청환", "녹창",
    "에친", "효보", "에보", "산청", "녹청", "에신", "자경", "친자", "산지", "해지",
    "자지", "에경", "에정", "재개", "녹개", "순보", "에순", "친합", "자합", "녹합"
]

negative_words_100 = [
    "낭비", "과다", "무분", "상시", "불필", "과도", "과소", "오염", "폐기", "파괴",
    "배출", "독성", "부적", "비효", "무절", "자낭", "남용", "무용", "소음", "미세",
    "화석", "비재", "독물", "온실", "과열", "불균", "불안", "폐물", "에낭", "무계",
    "파적", "해로", "위협", "불안", "부실", "부정", "자형", "부실운", "오남", "무효",
    "독배", "폐증", "소과", "불친", "비도", "위험", "낭성", "비합", "대오", "수오",
    "토오", "소공", "오물", "해오", "기악", "불투", "산폐", "산파", "자파", "대질",
    "자남", "화출", "유해", "폐수", "악취", "기유", "생파", "피해", "쓰범", "미폭",
    "독폐", "불법", "불광", "비위", "무책", "폐부", "온과", "온난", "공해", "규미",
    "피증", "재불", "무추", "쓰문", "도열", "온변", "위기", "생행", "불합", "비불",
    "부퇴", "폐악", "불폐", "비낭", "비오", "과배", "부오", "비폐", "지오", "산오"
]

# ── 2. 추가 단어 설정 (사용자 정의) ──
additional_positive_words = ["신뢰", "품질"]   # 추가 긍정 단어
additional_negative_words = ["불만", "문제"]     # 추가 부정 단어

# 기본 리스트와 추가 단어를 합침
positive_words = positive_words_100 + additional_positive_words
negative_words = negative_words_100 + additional_negative_words

def lexicon_score(text, pos_list, neg_list):
    """
    텍스트 내 긍정 단어와 부정 단어 등장 횟수를 비교하여 점수를 반환합니다.
    """
    pos_count = sum(1 for word in pos_list if word in text)
    neg_count = sum(1 for word in neg_list if word in text)
    return pos_count - neg_count

# ── 3. CSV 파일 로드 및 정답 라벨(PN) 활용 ──
# 파일에는 최소한 'description'과 정답 라벨인 'PN' 컬럼이 있어야 합니다.
df = pd.read_csv("./bulkdata/descriptions_with_PN.csv")
print("데이터셋 컬럼:", df.columns)

# lexicon 기반 점수 계산 (추가 단어까지 반영)
df["lex_score"] = df["description"].apply(lambda x: lexicon_score(x, positive_words, negative_words))

# ── 4. SentenceTransformer 모델 로드 및 임베딩 기반 라벨 산출 ──
model = SentenceTransformer('distiluse-base-multilingual-cased')

# 긍정/부정 단어들의 임베딩 평균 계산 (모든 단어 사용)
pos_embs = [model.encode(word) for word in positive_words]
avg_pos_emb = np.mean(pos_embs, axis=0) if pos_embs else None

neg_embs = [model.encode(word) for word in negative_words]
avg_neg_emb = np.mean(neg_embs, axis=0) if neg_embs else None

def assign_label_sentence_transformers(text):
    """
    문장의 임베딩을 구한 후, 긍정/부정 평균 임베딩과의 코사인 유사도를 비교하여 라벨을 반환합니다.
    """
    text_emb = model.encode(text)
    pos_sim = cosine_similarity(text_emb.reshape(1, -1), avg_pos_emb.reshape(1, -1))[0][0] if avg_pos_emb is not None else 0
    neg_sim = cosine_similarity(text_emb.reshape(1, -1), avg_neg_emb.reshape(1, -1))[0][0] if avg_neg_emb is not None else 0
    return 1 if pos_sim > neg_sim else -1

def compute_final_label(row):
    """
    lexicon 기반 점수가 0이 아니면 해당 결과를 사용하고, 0이면 임베딩 유사도 비교 결과를 사용합니다.
    """
    if row["lex_score"] != 0:
        return 1 if row["lex_score"] > 0 else -1
    else:
        return assign_label_sentence_transformers(row["description"])

# lexicon 및 임베딩 기반으로 최종 라벨(PN_final) 산출
df["PN_final"] = df.apply(compute_final_label, axis=1)

# ── 5. 정답(PN)과 우리 방법(PN_final) 비교 평가 ──
print("== Lexicon/Embedding 기반 예측 결과와 정답(PN) 비교 ==")
print("전체 정확도: {:.2f}%".format(accuracy_score(df["PN"], df["PN_final"])*100))
print("분류 리포트:")
print(classification_report(df["PN"], df["PN_final"]))
print("혼동 행렬:")
print(confusion_matrix(df["PN"], df["PN_final"]))

# ── 6. 로지스틱 회귀 분류기 학습 (문장 임베딩을 이용, 정답 라벨 PN 사용) ──
descriptions = df["description"].tolist()
X_train = model.encode(descriptions)
y_train = df["PN"].values  # 정답 라벨 사용

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# ── 7. 분류기 평가 (학습 데이터에서 예시) ──
pred_train = clf.predict(X_train)
print("== 분류기 학습 결과 (학습 데이터 평가) ==")
print("전체 정확도: {:.2f}%".format(accuracy_score(y_train, pred_train)*100))
print("분류 리포트:")
print(classification_report(y_train, pred_train))

# ── 8. 예측 함수 정의 ──
def predict_sentiment(sentence):
    """
    입력된 문장에 대해 학습된 분류기를 이용해 감성 라벨(1: 긍정, -1: 부정)을 예측하여 반환합니다.
    
    Parameters:
        sentence (str): 예측할 문장
        
    Returns:
        int: 예측된 감성 라벨 (1 또는 -1)
    """
    embedding = model.encode([sentence])
    pred = clf.predict(embedding)
    return int(pred[0])

# ── 9. 테스트 예시 ──
test_sentence = "이 제품은 정말 효율적이고 환경에도 좋아요."
result = predict_sentiment(test_sentence)
print("테스트 문장:", test_sentence)
print("예측 결과:", result)


데이터셋 컬럼: Index(['description', 'score', 'PN'], dtype='object')
== Lexicon/Embedding 기반 예측 결과와 정답(PN) 비교 ==
전체 정확도: 56.36%
분류 리포트:
              precision    recall  f1-score   support

          -1       0.69      0.57      0.62        35
           1       0.42      0.55      0.48        20

    accuracy                           0.56        55
   macro avg       0.56      0.56      0.55        55
weighted avg       0.59      0.56      0.57        55

혼동 행렬:
[[20 15]
 [ 9 11]]
== 분류기 학습 결과 (학습 데이터 평가) ==
전체 정확도: 67.27%
분류 리포트:
              precision    recall  f1-score   support

          -1       0.66      1.00      0.80        35
           1       1.00      0.10      0.18        20

    accuracy                           0.67        55
   macro avg       0.83      0.55      0.49        55
weighted avg       0.78      0.67      0.57        55

테스트 문장: 이 제품은 정말 효율적이고 환경에도 좋아요.
예측 결과: -1


In [33]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sentence_transformers import SentenceTransformer

# ── 1. 기본 단어 리스트 ──
positive_words_100 = [
    "절약", "효율", "친환", "재활", "보존", "절감", "최소", "지속", "절전", "저탄",
    "개선", "정화", "순환", "재생", "녹색", "청정", "산림", "해양", "자연", "중립",
    "다양", "에코", "혁신", "신재", "녹성", "회복", "도시", "경영", "투자", "정책",
    "자원", "소비", "의식", "스마트", "녹기", "물절", "친농", "자립", "실천", "행동",
    "지구", "환기", "환원", "감소", "조절", "에너", "산업", "녹산", "바람", "태양",
    "수력", "풍력", "전환", "저배", "녹전", "친건", "친소", "친화", "녹업", "신기",
    "산보", "해보", "친업", "전력", "녹율", "녹원", "친도", "녹지", "청원", "재순",
    "친생", "환산", "탄감", "녹실", "녹환", "에녹", "신환", "자환", "청환", "녹창",
    "에친", "효보", "에보", "산청", "녹청", "에신", "자경", "친자", "산지", "해지",
    "자지", "에경", "에정", "재개", "녹개", "순보", "에순", "친합", "자합", "녹합"
]

negative_words_100 = [
    "낭비", "과다", "무분", "상시", "불필", "과도", "과소", "오염", "폐기", "파괴",
    "배출", "독성", "부적", "비효", "무절", "자낭", "남용", "무용", "소음", "미세",
    "화석", "비재", "독물", "온실", "과열", "불균", "불안", "폐물", "에낭", "무계",
    "파적", "해로", "위협", "불안", "부실", "부정", "자형", "부실운", "오남", "무효",
    "독배", "폐증", "소과", "불친", "비도", "위험", "낭성", "비합", "대오", "수오",
    "토오", "소공", "오물", "해오", "기악", "불투", "산폐", "산파", "자파", "대질",
    "자남", "화출", "유해", "폐수", "악취", "기유", "생파", "피해", "쓰범", "미폭",
    "독폐", "불법", "불광", "비위", "무책", "폐부", "온과", "온난", "공해", "규미",
    "피증", "재불", "무추", "쓰문", "도열", "온변", "위기", "생행", "불합", "비불",
    "부퇴", "폐악", "불폐", "비낭", "비오", "과배", "부오", "비폐", "지오", "산오"
]

# ── 2. 추가 단어 설정 (사용자 정의) ──
additional_positive_words = ["신뢰", "품질"]   # 추가 긍정 단어
additional_negative_words = ["불만", "문제"]     # 추가 부정 단어

# 기본 리스트와 추가 단어를 합침
positive_words = positive_words_100 + additional_positive_words
negative_words = negative_words_100 + additional_negative_words

def lexicon_score(text, pos_list, neg_list):
    """
    텍스트 내 긍정 단어와 부정 단어 등장 횟수를 비교하여 점수를 반환합니다.
    """
    pos_count = sum(1 for word in pos_list if word in text)
    neg_count = sum(1 for word in neg_list if word in text)
    return pos_count - neg_count

# ── 3. CSV 파일 로드 및 정답 라벨(PN) 활용 ──
# 파일에는 'description'과 정답 라벨 'PN' 컬럼이 있어야 합니다.
df = pd.read_csv("./bulkdata/descriptions_with_PN.csv")
print("데이터셋 컬럼:", df.columns)

# lexicon 기반 점수 계산 (추가 단어까지 반영)
df["lex_score"] = df["description"].apply(lambda x: lexicon_score(x, positive_words, negative_words))

# ── 4. SentenceTransformer 모델 로드 및 임베딩 기반 라벨 산출 ──
model = SentenceTransformer('distiluse-base-multilingual-cased')

# 긍정/부정 단어들의 임베딩 평균 계산 (모든 단어 사용)
pos_embs = [model.encode(word) for word in positive_words]
avg_pos_emb = np.mean(pos_embs, axis=0) if pos_embs else None

neg_embs = [model.encode(word) for word in negative_words]
avg_neg_emb = np.mean(neg_embs, axis=0) if neg_embs else None

def assign_label_sentence_transformers(text):
    """
    문장의 임베딩을 구한 후, 긍정/부정 평균 임베딩과의 코사인 유사도를 비교하여 라벨을 반환합니다.
    """
    text_emb = model.encode(text)
    pos_sim = cosine_similarity(text_emb.reshape(1, -1), avg_pos_emb.reshape(1, -1))[0][0] if avg_pos_emb is not None else 0
    neg_sim = cosine_similarity(text_emb.reshape(1, -1), avg_neg_emb.reshape(1, -1))[0][0] if avg_neg_emb is not None else 0
    return 1 if pos_sim > neg_sim else -1

def compute_final_label(row):
    """
    lexicon 기반 점수가 0이 아니면 해당 결과를 사용하고, 0이면 임베딩 유사도 비교 결과를 사용합니다.
    """
    if row["lex_score"] != 0:
        return 1 if row["lex_score"] > 0 else -1
    else:
        return assign_label_sentence_transformers(row["description"])

# lexicon 및 임베딩 기반으로 최종 라벨(PN_final) 산출
df["PN_final"] = df.apply(compute_final_label, axis=1)

# ── 5. 정답(PN)과 우리 방법(PN_final) 비교 평가 ──
print("== Lexicon/Embedding 기반 예측 결과와 정답(PN) 비교 ==")
print("전체 정확도: {:.2f}%".format(accuracy_score(df["PN"], df["PN_final"]) * 100))
print("분류 리포트:")
print(classification_report(df["PN"], df["PN_final"]))
print("혼동 행렬:")
print(confusion_matrix(df["PN"], df["PN_final"]))

# ── 6. SVM 분류기 학습 (문장 임베딩을 이용, 정답 라벨 PN 사용) ──
descriptions = df["description"].tolist()
X_train = model.encode(descriptions)
y_train = df["PN"].values  # 정답 라벨 사용

# SVM 분류기 (여기서는 Linear kernel 사용)
svm_clf = SVC(kernel='linear', probability=True)
svm_clf.fit(X_train, y_train)

# ── 7. 분류기 평가 (학습 데이터 평가) ──
pred_train = svm_clf.predict(X_train)
print("== SVM 분류기 학습 결과 (학습 데이터 평가) ==")
print("전체 정확도: {:.2f}%".format(accuracy_score(y_train, pred_train) * 100))
print("분류 리포트:")
print(classification_report(y_train, pred_train))

# ── 8. 예측 함수 정의 ──
def predict_sentiment(sentence):
    """
    입력된 문장에 대해 학습된 SVM 분류기를 이용해 감성 라벨(1: 긍정, -1: 부정)을 예측하여 반환합니다.
    
    Parameters:
        sentence (str): 예측할 문장
        
    Returns:
        int: 예측된 감성 라벨 (1 또는 -1)
    """
    embedding = model.encode([sentence])
    pred = svm_clf.predict(embedding)
    return int(pred[0])

# ── 9. 테스트 예시 ──
test_sentence = "이 제품은 정말 효율적이고 환경에도 좋아요."
result = predict_sentiment(test_sentence)
print("테스트 문장:", test_sentence)
print("예측 결과:", result)


데이터셋 컬럼: Index(['description', 'score', 'PN'], dtype='object')
== Lexicon/Embedding 기반 예측 결과와 정답(PN) 비교 ==
전체 정확도: 56.36%
분류 리포트:
              precision    recall  f1-score   support

          -1       0.69      0.57      0.62        35
           1       0.42      0.55      0.48        20

    accuracy                           0.56        55
   macro avg       0.56      0.56      0.55        55
weighted avg       0.59      0.56      0.57        55

혼동 행렬:
[[20 15]
 [ 9 11]]
== SVM 분류기 학습 결과 (학습 데이터 평가) ==
전체 정확도: 67.27%
분류 리포트:
              precision    recall  f1-score   support

          -1       0.66      1.00      0.80        35
           1       1.00      0.10      0.18        20

    accuracy                           0.67        55
   macro avg       0.83      0.55      0.49        55
weighted avg       0.78      0.67      0.57        55

테스트 문장: 이 제품은 정말 효율적이고 환경에도 좋아요.
예측 결과: -1


In [40]:
import pandas as pd
from sentence_transformers import InputExample, SentenceTransformer, SentencesDataset, losses
from torch.utils.data import DataLoader
from datasets import Dataset
import builtins
builtins.Dataset = Dataset

# CSV 파일 로드 (예: 'description'과 정답 라벨 'PN'이 포함된 파일)
df = pd.read_csv("./bulkdata/descriptions_with_PN.csv")

# InputExample 객체 생성 (정답 라벨이 -1, 1인 경우, SoftmaxLoss를 위해 0, 1로 변환)
train_examples = []
for idx, row in df.iterrows():
    label = 0 if row['PN'] == -1 else 1
    train_examples.append(InputExample(texts=[row['description']], label=label))

# 사전 학습된 모델 로드
model_ft = SentenceTransformer('distiluse-base-multilingual-cased')

# SentencesDataset 및 DataLoader 준비
train_dataset = SentencesDataset(train_examples, model_ft)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

# 분류를 위한 SoftmaxLoss 설정
num_labels = 2  # 두 클래스: 0 (부정), 1 (긍정)
train_loss = losses.SoftmaxLoss(
    model=model_ft, 
    sentence_embedding_dimension=model_ft.get_sentence_embedding_dimension(), 
    num_labels=num_labels
)

# Fine tuning 실행
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)  # 약 10%의 warmup
model_ft.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    show_progress_bar=True
)

# fine tuning 후 모델을 사용하여 예측
example_sentence = "이 제품은 정말 효율적이고 환경에도 좋아요."
embedding = model_ft.encode(example_sentence)
print("Fine tuned embedding:", embedding[:5])  # 임베딩의 일부 출력


NameError: name 'DatasetDict' is not defined